In [ ]:
import gensim
import sys
sys.path.append('../Training/')
from res import *

In [ ]:
df=pd.read_csv('../Data/cleanTweets.csv',na_values={'None','NONE'})

df=df.replace(r'^\s*$', np.nan, regex=True)
df=df.fillna('')

In [ ]:
#loading SVM model
sl='../Training/learnWV8_1.sav' 
Llearn_obj=pickle.load(open(sl,'rb'))

#loading Word2Vec model
str_='../Training/w2v1_9_ESW5_C_swmod3_s_w2'
word2vec_path = str_+".bin"
word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)

In [ ]:
#This cell computes sentiment class for each tweet and corrosponding confidence score
import ast

def conf(arg):
    return max(arg[0],arg[1],arg[2])

df['tokens']=df.apply(lambda arg: ast.literal_eval(arg['tokens']),axis=1)

embed = get_word2vec_embeddings(word2vec, df)

df['pred']=Llearn_obj.predict(embed) #predicting sentiment class 
df['conf']=list(map(conf,Llearn_obj.predict_proba(embed))) #computing proability corrosponding to most confident prediction

In [ ]:
#conf restriction
d1=df.loc[df.index[df['pred']=='neu']]
d2=df.loc[df.index[df['pred']!='neu']]

#using only tweets with confident predictions
d1=d1.loc[d1.index[d1['conf']>=0.5]]#for neutral predictions use tweets with atleast 50% confidence (because support value for neutral is less)
d2=d2.loc[d2.index[d2['conf']>=0.6]]#for positive and negative predictions use tweets with atleast 60% confidence


d1=d1.reset_index(drop=True)
d2=d2.reset_index(drop=True)

df2=d1.append(d2,ignore_index=True)

df2=shuffle(df2)
df2=df2.reset_index(drop=True)
#df.to_csv('ProsAll_pred_prob2.csv',index=None,header=True) 

In [ ]:
df2.to_csv('../Data/cleanTweets_pred_prob.csv',index=None,header=True) 